In [3]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import shutil, os, time, datetime, re
from PIL import Image

In [2]:
!pip install pillow

  Using cached Pillow-8.0.1-cp39-cp39-win_amd64.whl (2.1 MB)


You should consider upgrading via the 'c:\users\pyris\onedrive\dokumente\projekte\programmieren\scraper\zfdg_website_scraper\env-win-bigm\scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
urljoin("https://www.laut.de/", "https://www.laut.de/bilder/__100,0/lautstark/artikel/cdreview/63423/cover,186495.jpg/bilder/__100,0/lautstark/artikel/cdreview/63423/cover,186495.jpg")

'https://www.laut.de/bilder/__100,0/lautstark/artikel/cdreview/63423/cover,186495.jpg/bilder/__100,0/lautstark/artikel/cdreview/63423/cover,186495.jpg'

In [9]:
from scraper import Analyzer

In [ ]:
ana = Analyzer()
images_dict = ana.getImages()